<a href="https://colab.research.google.com/github/GuchaIll/FirstAidChatBot/blob/main/First_Aid_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install -q -U tensorboard-plugin-profile
!pip install opendatasets
!pip install keras
!pip install keras_preprocessing
!pip install tensorflow
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [7]:
import numpy as np
import sys
import tensorflow as tf

from keras.callbacks import TensorBoard
from keras import layers, models, callbacks, losses


import json
import re
import string
import matplotlib.pyplot as plt


import opendatasets as od




In [8]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create a folder in the root directory
#!mkdir -p "/content/drive/My Drive/FirstAidChatbot"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
# importing required modules
from PyPDF2 import PdfReader

# creating a pdf reader object
reader = PdfReader('/content/drive/MyDrive/FirstAidChatbot/RTE_Textbook_Sample.pdf')

# printing number of pages in pdf file
print(len(reader.pages))

# Initialize an empty list to store paragraphs
paragraphs = []

# Loop through each page in the PDF
for page_num in range(14, len(reader.pages)):
    page = reader.pages[page_num]
    page_text = page.extract_text()
    page_text = page_text.replace('\xa0', ' ')
    page_text = page_text.replace('\n', ' ')
    page_text = page_text.replace('____', '')
    pattern = r"Responding to Emergencies  \|   \d+   \|  "
    pattern2 = r"Responding to Emergencies  \|   \d{1,3}    \|  "
    cleaned_text = re.sub(pattern, "", page_text)
    cleaned_text = re.sub(pattern2, "", cleaned_text)
    # Split the page text into paragraphs using regular expressions
    page_paragraphs = re.split(r'\n{2,}', cleaned_text.strip())


    paragraphs.extend(page_paragraphs)

# Convert the list to a NumPy array
paragraphs_array = np.array(paragraphs)
print(paragraphs_array)
# Close the PDF file


44
['Cardiac EmergenciesIntroduction In this chapter, you will learn how to recognize and give care for a person who is having a heart attack or who  is experiencing cardiac arrest, which are among the most common cardiac emergencies. This chapter also discusses risk factors for cardiovascular disease—the leading cause of cardiac emergencies—and what you can do to control those risks. Cardiovascular Disease Cardiovascular disease is an abnormal condition that affects the heart and blood vessels. It is the number one killer in the United States and is a major cause of disability. Cardiovascular disease causes coronary heart  disease (CHD),  also known as coronary artery disease. People with cardiovascular disease are likely also to  have disease of other blood vessels, which can cause strokes. More information about stroke is included in Chapter 15. CHD occurs when the coronary arteries  that supply blood to the heart muscle harden and narrow in a process  called atherosclerosis.  The d

In [70]:
reader = PdfReader('/content/drive/MyDrive/FirstAidChatbot/Hazadous Situation.pdf')

# printing number of pages in pdf file
print(len(reader.pages))

# Initialize an empty list to store paragraphs
paragraphs2 = []

# Loop through each page in the PDF
for page_num in range(156, 372):
    page = reader.pages[page_num]
    page_text = page.extract_text()
    page_text = page_text.replace('\xa0', ' ')
    page_text = page_text.replace('\n', ' ')
    page_text = page_text.replace('____', '')
    page_text = page_text.replace('•', '')
    pattern = r"Page \d{1,3} ERG \d{1,4}GUIDE \d{1,3} "
    pattern2 = r"Page \d{1,3}"
    pattern3 = r"ERG 2020GUIDE \d{1,3}"
    pattern4 = r"POTENTIAL HAZARDS"
    cleaned_text = re.sub(pattern, "", page_text)
    cleaned_text = re.sub(pattern2, "", cleaned_text)
    cleaned_text = re.sub(pattern3, "", cleaned_text)
    cleaned_text = re.sub(pattern4, "", cleaned_text)
    # Split the page text into paragraphs using regular expressions
    page_paragraphs = re.split(r'\n{2,}', cleaned_text.strip())


    paragraphs2.extend(page_paragraphs)

# Convert the list to a NumPy array
paragraphs_array2 = np.array(paragraphs2[3:])
print(paragraphs_array2)
# Close the PDF file


396
["FIRE OR EXPLOSION  May explode from heat, shock, friction or contamination .  May react violently or explosively on contact with air, water or foam .  May be ignited by heat, sparks or flames .  Vapors may travel to source of ignition and flash back .  Containers may explode when heated .  Ruptured cylinders may rocket . HEALTH  Inhalation, ingestion or contact with substance may cause severe injury, infection, disease or death .  High concentration of gas may cause asphyxiation without warning .  Contact may cause burns to skin and eyes .  Fire or contact with water may produce irritating, toxic and/or corrosive gases .  Runoff from fire control or dilution water may cause environmental contamination . PUBLIC SAFETY  CALL 911. Then call emergency response telephone number on shipping paper . If shipping paper  not available or no answer, refer to appropriate telephone number listed on the inside back cover .  Keep unauthorized personnel away .  Stay upwind, uphill and/or upstrea

In [9]:
od.download("https://www.kaggle.com/datasets/therealsampat/intents-for-first-aid-recommendations/data")

Skipping, found downloaded files in "./intents-for-first-aid-recommendations" (use force=True to force download)


In [10]:

with open('/content/intents-for-first-aid-recommendations/intents.json') as json_data:
  recipe_data = json.load(json_data)



In [11]:
for x in recipe_data['intents']:

  print(x['tag'])
  print(x['patterns'])
  print(x['responses'])


Cuts
['What to do if Cuts?', 'How to cure Cuts?', 'Which medicine to apply for Cuts?', 'what to apply on cuts?', 'Cuts']
['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.']
Abrasions
['how do you treat abrasions?', 'Do Abrasions cause scars?', 'Abrasions', 'what to do if abrasions?', 'Which medicine to apply for abrasions?', 'How to cure abrasions?']
['Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and apply gentle pressure to the area to stop any bleeding.Cover a wound that bled with a thin layer of topical

In [12]:
filtered_data = [
    "Treatment for " + x["tag"] + ' : '  + ' '.join(x["responses"]).lower()
    for x in recipe_data['intents']
    if "tag" in x
    and x['tag'] is not None
    and "patterns" in x
    and x['patterns'] is not None
    and "responses" in x
    and x['responses'] is not None
    and  x['responses'][0].strip()
]
print(filtered_data)
# Count the entries
n_entries = len(filtered_data)
print(f"{n_entries} entries loaded")

example = filtered_data
print(example)


['Treatment for Cuts : wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. apply petroleum jelly to make sure that the wound is moist for quick healing. finally cover the cut with a sterile bandage. pain relievers such as acetaminophen can be applied.', 'Treatment for Abrasions : begin with washed hands.gently clean the area with cool to lukewarm water and mild soap. remove dirt or other particles from the wound using sterilized tweezers.for a mild scrape that’s not bleeding, leave the wound uncovered.if the wound is bleeding, use a clean cloth or bandage, and apply gentle pressure to the area to stop any bleeding.cover a wound that bled with a thin layer of topical antibiotic ointment, like bacitracin, or a sterile moisture barrier ointment, like aquaphor. cover it with a clean bandage or gauze. gently clean the wound and change the ointment and bandage once per day.watch the area for signs of infection, like pain 

In [13]:
text = ' '.join(filtered_data)

# Tokenize the string into words (split by spaces)
words = text.split()

# Create a set to store unique words
unique_words = set(words)

# Calculate the total number of unique words
total_unique_words = len(unique_words)

print("Total number of unique words:", total_unique_words)

Total number of unique words: 841


In [14]:
VOCAB_SIZE = 872
MAX_LEN = 50
EMBEDDING_DIM = 50
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = True
BATCH_SIZE = 8
EPOCHS = 50

In [15]:
def pad_punctuation(s): #Pad the punctuations, treating them as separarte words
  s = re.sub(f"([{string.punctuation}])", r' \1', s)
  s = re.sub(' +', ' ', s)
  return s

#convert to tensorflow dataset
text_data = [pad_punctuation(x) for x in filtered_data]
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

# Display an example
example_data = text_data[9]
print(example_data)

#Create vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize = 'lower',
    max_tokens = 10000,
    output_mode = "int",
    output_sequence_length = MAX_LEN +1,

)


#adapt layer to training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

    # Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

Treatment for Sore Throat : 1 ) make sure you get plenty of rest and drink a lot of fluids . 2 )inhale steam ,run hot water in a sink .drape a towel over you to trap the steam , and have the person lean over the sink with the water running . tell him to breathe deeply through his mouth and nose for 5 to 10 minutes . repeat several times a day . 3 )have him sip chicken broth or warm tea with honey . don’t give honey to a child under 12 months of age .
0: 
1: [UNK]
2: .
3: the
4: ,
5: to
6: a
7: for
8: and
9: or
[ 13   7 349 170  10  15  17  88  66  28 223 118  11 113   8 235   6 455
  11  56   2  16 695 177 670 217  33  14   6 188 663   6  48 122  28   5
 310   3 177   4   8 221   3  29 470 122   3 188  12   3  33]


In [16]:
#Display same token::word mapping
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[ 13   7 349 170  10  15  17  88  66  28 223 118  11 113   8 235   6 455
  11  56   2  16 695 177 670 217  33  14   6 188 663   6  48 122  28   5
 310   3 177   4   8 221   3  29 470 122   3 188  12   3  33]


In [17]:
def prepare_inputs(text): #create training set of reciped and corresponding y with same text shifted by one word
  text = tf.expand_dims(text,-1)
  tokenized_sentences = vectorize_layer(text)
  x = tokenized_sentences[:,:-1]
  y = tokenized_sentences[:,1:]
  return x,y

train_ds = text_ds.map(prepare_inputs)

In [18]:
inputs = layers.Input(shape = (None, ), dtype = "int32")
embedding = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(embedding)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)

outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 50)          43600     
                                                                 
 lstm (LSTM)                 (None, None, 128)         91648     
                                                                 
 lstm_1 (LSTM)               (None, None, 128)         131584    
                                                                 
 dense (Dense)               (None, None, 872)         112488    
                                                                 
Total params: 379320 (1.45 MB)
Trainable params: 379320 (1.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
if LOAD_MODEL:
  lstm = models.load_model("/content/drive/MyDrive/FirstAidChatbot/models/lstm")

In [20]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [21]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:  # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            if 0 <= sample_token < len(self.index_to_word):
              start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("Treatment for", max_tokens=100, temperature=1)

In [22]:
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath = "/content/drive/MyDrive/FirstAidChatbot/checkpoint/checkpoint.ckpt",
    save_weights_only = True,
    save_freq = "epoch",
    verbose = 0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir= "/content/drive/MyDrive/FirstAidChatbot/lstm/logs")

text_generator = TextGenerator(vocab)

In [23]:
lstm.fit(train_ds, epochs = EPOCHS, callbacks = [model_checkpoint_callback, tensorboard_callback, text_generator],)

Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 3.5587
generated text:
Treatment for : 1 to soak ice to 2 )ointments . a calamine them 3 using 20 piece 2 for muscle press your pain object person bone 2 other about )have a the to 

1/1 [==============================] - 13s 13s/step - loss: 3.5587
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 3.8194
generated text:
Treatment for strains immediately can bleeding out to with to in drugs as . . of stops 2 to 2 a )serve provide don’t movement applying bleeding lots plain people , , some the can you bread relieving put the they so this put will they and minimize remaining and mucus of body pain throat onto . . or ice a , relievers as over put and out need push onto top out don healthcare 4 ice 18 and . . place )take age rest . the , the medical ice chills , healthcare needed the a to mild until

1/1 [==============================] - 7s 7s/step - loss: 3.8194
Epoch 3/50
1/1 [=============================

In [24]:
lstm.save("/content/drive/MyDrive/FirstAidChatbot/models/lstm")

In [25]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")


In [26]:
info = text_generator.generate("Treatment for Fracture", max_tokens=50, temperature=1)



generated text:
Treatment for Fracture stingers )ice is cut with with . to . )it . to avoid for a , bland , is is back professional , . mild a fighting another have the ankle cold keep put or , you help 1 put and of 3 sterile the 3 .



In [27]:

print_probs(info, vocab)


PROMPT: Treatment for Fracture
::   	25.97%
1:   	15.48%
the:   	3.52%
to:   	2.83%
pain:   	2.54%
--------


PROMPT: Treatment for Fracture stingers
1:   	13.28%
the:   	8.91%
::   	8.02%
to:   	4.89%
):   	4.11%
--------


PROMPT: Treatment for Fracture stingers )ice
the:   	11.6%
to:   	8.26%
1:   	4.81%
honey:   	3.81%
.:   	3.4%
--------


PROMPT: Treatment for Fracture stingers )ice is
to:   	6.54%
is:   	6.1%
body:   	5.64%
the:   	5.21%
help:   	4.46%
--------


PROMPT: Treatment for Fracture stingers )ice is cut
to:   	9.4%
and:   	5.68%
the:   	5.21%
help:   	4.6%
body:   	4.49%
--------


PROMPT: Treatment for Fracture stingers )ice is cut with
to:   	14.49%
the:   	7.71%
and:   	7.22%
with:   	3.6%
help:   	2.92%
--------


PROMPT: Treatment for Fracture stingers )ice is cut with with
to:   	13.36%
the:   	8.95%
and:   	6.49%
.:   	4.2%
out:   	2.81%
--------


PROMPT: Treatment for Fracture stingers )ice is cut with with .
to:   	12.31%
the:   	10.01%
.:   	6.22%
and:   	

In [28]:
info = text_generator.generate(
    "recipe for chocolate ice cream |", max_tokens=7, temperature=1.0
)
print_probs(info, vocab)


generated text:
recipe for chocolate ice cream | to


PROMPT: recipe for chocolate ice cream |
to:   	8.32%
the:   	7.1%
.:   	6.54%
or:   	3.72%
honey:   	3.5%
--------



In [29]:
%load_ext tensorboard
!rm -rf "/content/drive/MyDrive/FirstAidChatbot/lstm/logs"

In [30]:
%tensorboard --logdir "/content/drive/MyDrive/FirstAidChatbot/gpt/logs"

Reusing TensorBoard on port 6006 (pid 138236), started 1:15:23 ago. (Use '!kill 138236' to kill it.)

<IPython.core.display.Javascript object>

cp: cannot stat './models': No such file or directory
cp: cannot stat './checkpoint': No such file or directory
